In [7]:
from settrade_v2 import Investor
from pprint import pprint
from settrade_v2 import Investor
import psycopg2
import pandas as pd
import time
from psycopg2 import sql
from datetime import datetime
from pprint import pprint
import mplfinance as mpf
import matplotlib.pyplot as plt
import numpy as np

In [8]:
investor = Investor(
                    app_id="xzxY3IciHakYITRk",
                    app_secret="AMPs/EPM3jIYeNlY2otoMcQawPFBcn21p1XR5sze+z9Y",
                    broker_id="SANDBOX",
                    app_code="SANDBOX",
                    is_auto_queue = False)

In [9]:
# กรณี Investor
market = investor.MarketData()

In [10]:
mkt_data = investor.MarketData()
res = mkt_data.get_quote_symbol("AOT")
pprint(res)

{'aumSize': None,
 'average': None,
 'change': None,
 'eps': 0.37,
 'exchange': None,
 'exercisePrice': None,
 'exerciseRatio': None,
 'high': None,
 'impliedVolatility': None,
 'inav': None,
 'instrumentType': 'STOCK',
 'intrinsicValue': None,
 'last': None,
 'lastTradingDate': None,
 'low': None,
 'marketStatus': 'Close',
 'maturityDate': None,
 'moneyness': None,
 'pbv': 4.41,
 'pe': 28.62,
 'percentChange': None,
 'percentYield': 1.9800000190734863,
 'securityType': 'CS',
 'status': '',
 'symbol': 'AOT',
 'theoretical': None,
 'toLastTrade': None,
 'totalBuyVolume': 276600,
 'totalNoSideVolume': 93400,
 'totalSellVolume': 76800,
 'totalVolume': 0,
 'underlying': None,
 'underlyingPrice': None}


In [6]:
def insert_financial_data(symbol):
    try:
        print(f"🔍 กำลังดึงข้อมูล Financial Data สำหรับ {symbol}...")
        res = market.get_quote_symbol(symbol)

        # 🚨 ตรวจสอบว่ามีข้อมูลที่ถูกต้อง
        if not isinstance(res, dict) or "symbol" not in res:
            print(f"⚠️ ไม่มีข้อมูลสำหรับ {symbol}. Response: {res}")
            return

        # 📝 แสดงค่าที่ได้รับจาก API
        print(f"✅ Data from API for {symbol}:")
        for key, value in res.items():
            print(f"{key}: {value}")

    except Exception as e:
        print(f"❌ เกิดข้อผิดพลาด: {e}")


In [7]:
print(res)

{'instrumentType': 'STOCK', 'symbol': 'AOT', 'high': None, 'low': None, 'last': None, 'average': None, 'change': None, 'percentChange': None, 'totalVolume': 0, 'marketStatus': 'Close', 'securityType': 'CS', 'eps': None, 'pe': None, 'pbv': None, 'percentYield': None, 'maturityDate': None, 'exercisePrice': None, 'underlying': None, 'underlyingPrice': None, 'intrinsicValue': None, 'theoretical': None, 'moneyness': None, 'lastTradingDate': None, 'toLastTrade': None, 'exerciseRatio': None, 'impliedVolatility': None, 'exchange': None, 'aumSize': None, 'inav': None}


In [ ]:
print(f"🔹 SQL Query: {insert_query.as_string(conn)}")
print(f"🔹 Data: {data}")


NameError: name 'insert_query' is not defined

In [ ]:
db_config = {
    'dbname': 'Financial_data',
    'user': 'postgres',
    'password': 'admin',
    'host': 'localhost',
    'port': "5432"
}

In [ ]:
# ✅ ฟังก์ชันดึงข้อมูล Financial Data จาก SETTRADE API และบันทึกลง PostgreSQL
def insert_financial_data(symbol):
    try:
        print(f"🔍 กำลังดึงข้อมูล Financial Data สำหรับ {symbol}...")
        res = market.get_quote_symbol(symbol)

        # 🚨 ถ้าไม่มีข้อมูล ให้ข้าม
        if not isinstance(res, dict) or "symbol" not in res:
            print(f"⚠️ ไม่มีข้อมูลสำหรับ {symbol}.")
            return

        # 🔧 กำหนดค่า None เป็น 0 หรือค่าเริ่มต้น
        data = {
            'symbol': res.get('symbol', symbol),
            'instrument_type': res.get('instrumentType', 'STOCK'),
            'security_type': res.get('securityType', 'CS'),
            'market_status': res.get('marketStatus', 'Unknown'),
            'total_volume': res.get('totalVolume', 0),
            'aum_size': res.get('aumSize', 0),
            'average': res.get('average', 0),
            'change': res.get('change', 0),
            'eps': res.get('eps', 0),
            'exchange': res.get('exchange', ''),
            'exercise_price': res.get('exercisePrice', 0),
            'exercise_ratio': res.get('exerciseRatio', 0),
            'high': res.get('high', 0),
            'implied_volatility': res.get('impliedVolatility', 0),
            'inav': res.get('inav', 0),
            'intrinsic_value': res.get('intrinsicValue', 0),
            'last': res.get('last', 0),
            'last_trading_date': res.get('lastTradingDate', ''),
            'low': res.get('low', 0),
            'maturity_date': res.get('maturityDate', ''),
            'moneyness': res.get('moneyness', 0),
            'pbv': res.get('pbv', 0),
            'pe': res.get('pe', 0),
            'percent_change': res.get('percentChange', 0),
            'percent_yield': res.get('percentYield', 0),
            'theoretical': res.get('theoretical', 0),
            'to_last_trade': res.get('toLastTrade', 0),
            'underlying': res.get('underlying', ''),
            'underlying_price': res.get('underlyingPrice', 0),
            'updated_at': 'NOW()'
        }

        # 🔗 เชื่อมต่อฐานข้อมูล
        conn = psycopg2.connect(**db_config)
        cursor = conn.cursor()

        # 📌 คำสั่ง SQL สำหรับ INSERT
        insert_query = sql.SQL("""
            INSERT INTO stock_financials (
                symbol, instrument_type, security_type, market_status, total_volume, 
                aum_size, average, change, eps, exchange, exercise_price, 
                exercise_ratio, high, implied_volatility, inav, intrinsic_value, 
                last, last_trading_date, low, maturity_date, moneyness, pbv, pe, 
                percent_change, percent_yield, theoretical, to_last_trade, underlying, 
                underlying_price, updated_at
            ) VALUES (
                %(symbol)s, %(instrument_type)s, %(security_type)s, %(market_status)s, %(total_volume)s,
                %(aum_size)s, %(average)s, %(change)s, %(eps)s, %(exchange)s, %(exercise_price)s,
                %(exercise_ratio)s, %(high)s, %(implied_volatility)s, %(inav)s, %(intrinsic_value)s,
                %(last)s, %(last_trading_date)s, %(low)s, %(maturity_date)s, %(moneyness)s, %(pbv)s, %(pe)s,
                %(percent_change)s, %(percent_yield)s, %(theoretical)s, %(to_last_trade)s, %(underlying)s,
                %(underlying_price)s, NOW()
            )
            ON CONFLICT (symbol) DO UPDATE SET
                instrument_type = EXCLUDED.instrument_type,
                security_type = EXCLUDED.security_type,
                market_status = EXCLUDED.market_status,
                total_volume = EXCLUDED.total_volume,
                aum_size = EXCLUDED.aum_size,
                average = EXCLUDED.average,
                change = EXCLUDED.change,
                eps = EXCLUDED.eps,
                exchange = EXCLUDED.exchange,
                exercise_price = EXCLUDED.exercise_price,
                exercise_ratio = EXCLUDED.exercise_ratio,
                high = EXCLUDED.high,
                implied_volatility = EXCLUDED.implied_volatility,
                inav = EXCLUDED.inav,
                intrinsic_value = EXCLUDED.intrinsic_value,
                last = EXCLUDED.last,
                last_trading_date = EXCLUDED.last_trading_date,
                low = EXCLUDED.low,
                maturity_date = EXCLUDED.maturity_date,
                moneyness = EXCLUDED.moneyness,
                pbv = EXCLUDED.pbv,
                pe = EXCLUDED.pe,
                percent_change = EXCLUDED.percent_change,
                percent_yield = EXCLUDED.percent_yield,
                theoretical = EXCLUDED.theoretical,
                to_last_trade = EXCLUDED.to_last_trade,
                underlying = EXCLUDED.underlying,
                underlying_price = EXCLUDED.underlying_price,
                updated_at = NOW();
        """)

        # 🚀 บันทึกข้อมูลลง PostgreSQL
        cursor.execute(insert_query, data)
        conn.commit()
        print(f"✅ ข้อมูล Financial Data ของ {symbol} ถูกบันทึกเรียบร้อยแล้ว!")

    except Exception as e:
        print(f"❌ เกิดข้อผิดพลาดกับ {symbol}: {e}")

    finally:
        if 'cursor' in locals(): cursor.close()
        if 'conn' in locals(): conn.close()


In [ ]:
# 🔥 โหลดรายชื่อหุ้นทั้งหมดจากไฟล์ CSV
try:
    symbols_df = pd.read_csv("thai_stocks.csv")  # ไฟล์ต้องมีคอลัมน์ 'symbol'
    symbols = symbols_df['symbol'].dropna().tolist()
except FileNotFoundError:
    print("❌ ไม่พบไฟล์ 'thai_stocks.csv' ใช้รายชื่อหุ้นที่กำหนดแทน")
    symbols = ["PTT", "AOT", "SCB", "CPALL", "ADVANC"]

# 🔄 ดึงข้อมูล Financial Data สำหรับหุ้นทุกตัว
for symbol in symbols:
    insert_financial_data(symbol)
    time.sleep(2)  # ⏳ ลด API Rate Limit


🔍 กำลังดึงข้อมูล Financial Data สำหรับ 24CS...
✅ ข้อมูล Financial Data ของ 24CS ถูกบันทึกเรียบร้อยแล้ว!
🔍 กำลังดึงข้อมูล Financial Data สำหรับ 2S...
✅ ข้อมูล Financial Data ของ 2S ถูกบันทึกเรียบร้อยแล้ว!
🔍 กำลังดึงข้อมูล Financial Data สำหรับ 3BBIF...
✅ ข้อมูล Financial Data ของ 3BBIF ถูกบันทึกเรียบร้อยแล้ว!
🔍 กำลังดึงข้อมูล Financial Data สำหรับ A...
✅ ข้อมูล Financial Data ของ A ถูกบันทึกเรียบร้อยแล้ว!
🔍 กำลังดึงข้อมูล Financial Data สำหรับ A5...
✅ ข้อมูล Financial Data ของ A5 ถูกบันทึกเรียบร้อยแล้ว!
🔍 กำลังดึงข้อมูล Financial Data สำหรับ AAI...
✅ ข้อมูล Financial Data ของ AAI ถูกบันทึกเรียบร้อยแล้ว!
🔍 กำลังดึงข้อมูล Financial Data สำหรับ AAV...
✅ ข้อมูล Financial Data ของ AAV ถูกบันทึกเรียบร้อยแล้ว!
🔍 กำลังดึงข้อมูล Financial Data สำหรับ ABM...
✅ ข้อมูล Financial Data ของ ABM ถูกบันทึกเรียบร้อยแล้ว!
🔍 กำลังดึงข้อมูล Financial Data สำหรับ ACAP...
✅ ข้อมูล Financial Data ของ ACAP ถูกบันทึกเรียบร้อยแล้ว!
🔍 กำลังดึงข้อมูล Financial Data สำหรับ ACC...
✅ ข้อมูล Financial Data ของ ACC ถูกบ

KeyboardInterrupt: 